In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
import os
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

# Set image dimensions for MobileNetV3
IMG_SIZE = (224, 224)

# Load MobileNetV3 (pre-trained on ImageNet, exclude top layers)
model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

# Function to load and preprocess MRI images
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=IMG_SIZE)  # Load and resize image
    img_array = image.img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = preprocess_input(img_array)  # Preprocess as per MobileNetV3
    return img_array

# Extract features from MRI images using MobileNetV3
def extract_features(image_paths):
    features = []
    for img_path in image_paths:
        img_array = load_and_preprocess_image(img_path)
        features.append(model.predict(img_array).flatten())  # Flatten the features
    return np.array(features)

# Paths to stroke and normal MRI images (update these paths)
stroke_image_paths = [os.path.join(r'C:\Users\soura\archive (1)\Acute Ischemic Stroke-Control\Acute Ischemic Stroke', img)
                      for img in os.listdir(r'C:\Users\soura\archive (1)\Acute Ischemic Stroke-Control\Acute Ischemic Stroke')]

normal_image_paths = [os.path.join(r'C:\Users\soura\archive (1)\Acute Ischemic Stroke-Control\Control', img)
                      for img in os.listdir(r'C:\Users\soura\archive (1)\Acute Ischemic Stroke-Control\Control')]

# Extract features using MobileNetV3
stroke_features = extract_features(stroke_image_paths)
normal_features = extract_features(normal_image_paths)

# Labels: 1 for stroke, 0 for normal
labels = np.concatenate([np.ones(len(stroke_features)), np.zeros(len(normal_features))])

# Combine features and labels
X = np.concatenate([stroke_features, normal_features])
y = labels

# Apply PCA for dimensionality reduction
pca = PCA(n_components=300)  # Experiment with the number of components
X_pca = pca.fit_transform(X)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

# Tuning LightGBM using GridSearchCV
lgb_model = lgb.LGBMClassifier()
lgb_params = {
    'num_leaves': [31, 40],
    'learning_rate': [0.001, 0.005,],
    'n_estimators': [100, 200]
}
lgb_grid = GridSearchCV(lgb_model, lgb_params, cv=5)
lgb_grid.fit(X_train, y_train)
lgb_best = lgb_grid.best_estimator_

# Tuning CatBoost using hyperparameters
catboost_model = CatBoostClassifier(verbose=0, depth=6, learning_rate=0.05, iterations=500)
catboost_model.fit(X_train, y_train)

# Predict with LightGBM and CatBoost on training data
lgb_predictions_train = lgb_best.predict(X_train)
catboost_predictions_train = catboost_model.predict(X_train)

# Stack predictions from the base models
stacked_predictions_train = np.column_stack((lgb_predictions_train, catboost_predictions_train))

# Train a meta-model (Logistic Regression) on the stacked predictions
meta_model = LogisticRegression()
meta_model.fit(stacked_predictions_train, y_train,)

# Test Phase: Generate predictions
lgb_predictions_test = lgb_best.predict(X_test)
catboost_predictions_test = catboost_model.predict(X_test)

# Stack predictions on test data
stacked_predictions_test = np.column_stack((lgb_predictions_test, catboost_predictions_test))

# Final predictions using the meta-model
final_predictions = meta_model.predict(stacked_predictions_test)

# Evaluate the final model
accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Model Accuracy: {accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━